## Separação de páginas de acordo com o coletor necessário

Dado uma página inicial de portal de transparência de uma prefeitura, é de interesse que possamos gerar links a partir dela e separar essses links gerados de acordo com subtags de dados a serem coletados (o que é tarefa para um classificador de páginas) e principalmente considerando o coletor necessário para recuperação da informação (tema abordado nesse notebook). A hipótese inicial é que isso pode ser feito comparando e clusterizando as páginas de acordo com sua estrutura, seu estilo ou considerando os botões presentes nela.

In [32]:
!pip install jellyfish
!pip install html-similarity

In [3]:
import urllib3
import jellyfish
import requests

from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib3.exceptions import *
from html_similarity import style_similarity, structural_similarity, similarity

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-web-security")
options.add_argument('--headless')

A seguir, há algumas páginas do Template 2. Essas páginas foram agrupadas de acordo com os coletores necessários para cada grupo, excetuando-se a paginas_extras, que se tratam de páginas isoladas do template. O ideal é que páginas de mesmo grupo (excetuando-se o último) apresentem resultados bem proximos entre si e resultados distantes de páginas de outros grupos. Assim, para que se possa mensurar isso, testou-se utilizando diversas métricas.

In [34]:
paginas_base = ["https://freigaspar.mg.gov.br/transparencia/empenhos", 
                "https://freigaspar.mg.gov.br/transparencia/pagamentos",
                "https://freigaspar.mg.gov.br/transparencia/receitas"]

coletor_1 = ["https://freigaspar.mg.gov.br/transparencia/folhas-de-pagamento/detalhes?indSituacaoServidorPensionista%5B0%5D=P&indSituacaoServidorPensionista%5B1%5D=03&IDE_FLPGO_id=35&ano=2015",
             "https://freigaspar.mg.gov.br/transparencia/empenhos/detalhes/2022/02/118",
             "https://freigaspar.mg.gov.br/transparencia/pagamentos/detalhes/2022/02/118"]

coletor_2 = ["https://freigaspar.mg.gov.br/transparencia/empenhos/exibir/2022/02/33089",
             "https://freigaspar.mg.gov.br/transparencia/pagamentos/exibir/2021/12/35798",
             "https://freigaspar.mg.gov.br/transparencia/empenhos/exibir/2022/05/34915"]

paginas_extras = ["https://freigaspar.mg.gov.br/transparencia",
                  "https://freigaspar.mg.gov.br/transparencia/folhas-de-pagamento",
                  "https://freigaspar.mg.gov.br/transparencia/coronavirus"]

testes = paginas_base + coletor_1 + coletor_2 + paginas_extras

['https://freigaspar.mg.gov.br/transparencia/empenhos', 'https://freigaspar.mg.gov.br/transparencia/pagamentos', 'https://freigaspar.mg.gov.br/transparencia/receitas', 'https://freigaspar.mg.gov.br/transparencia/folhas-de-pagamento/detalhes?indSituacaoServidorPensionista%5B0%5D=P&indSituacaoServidorPensionista%5B1%5D=03&IDE_FLPGO_id=35&ano=2015', 'https://freigaspar.mg.gov.br/transparencia/empenhos/detalhes/2022/02/118', 'https://freigaspar.mg.gov.br/transparencia/pagamentos/detalhes/2022/02/118', 'https://freigaspar.mg.gov.br/transparencia/empenhos/exibir/2022/02/33089', 'https://freigaspar.mg.gov.br/transparencia/pagamentos/exibir/2021/12/35798', 'https://freigaspar.mg.gov.br/transparencia/empenhos/exibir/2022/05/34915', 'https://freigaspar.mg.gov.br/transparencia', 'https://freigaspar.mg.gov.br/transparencia/folhas-de-pagamento', 'https://freigaspar.mg.gov.br/transparencia/coronavirus']


Assim, geram-se matrizes de similaridade de acordo com cada uma das métricas consideradas. Nessa matriz, as submatrizes 3x3 que começam nos elementos [0][0], [3][3] e [6][6] idealmente devem possuir valores próximos e que indiquem alta semelhança, o que é um sinal de que um mesmo coletor pode ser utilizado.

In [85]:
RED   = "\033[1;31m"  
BLUE  = "\033[1;34m"
CYAN  = "\033[1;36m"
GREEN = "\033[0;32m"
RESET = "\033[0;0m" 

In [86]:
def gerarResultados(metrica, dados):
    
    similares = 0
    nao_similares = 0
    
    for i in range(len(dados)):
        for j in range(len(dados)):
            resultado = metrica(dados[i], dados[j])
            string = f"{resultado:.2f}"
            if i<3 and j< 3:
                print(GREEN + string, end = " ")
                similares += resultado
            elif i<6 and i>2 and j<6 and j>2:
                print(BLUE + string, end = " ")
                similares += resultado
            elif i<9 and i>5 and j<9 and j>5:
                print(CYAN + string, end = " ")
                similares += resultado
            else:
                print(RED + string, end = " ")
                nao_similares += resultado
        print("\n")
        
    print(RESET + "Similaridade Desejada Média = " + f"{(similares/27):.2f}")
    print("Similaridade Não Desejada Média = " + f"{(nao_similares/117):.2f}")

## Similaridade entre strings usando Jellyfish

In [78]:
def extrairHtml(url):

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
    try:
        soup = BeautifulSoup(driver.page_source, "lxml")
        
        driver.close()
        
        for elm in soup.find_all():
            if not elm.find(recursive=False):
                elm.string = ''
            elm.attrs = {}
            
        html = str(soup.prettify()).replace("\n", "")
            
        return html.replace(" ", "")

    
    except:
        driver.close()
        
        print("Erro ao tentar abrir a pagina: " + url)

In [6]:
extrairHtml("https://freigaspar.mg.gov.br/transparencia/coronavirus")

'<html><head><!--|METAS|--><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><!--|OGTAGS|--><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><meta></meta><link></link><title></title><style></style><!--|CSS|--><link></link><link></link><link></link><link></link><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style><style></style></head><body><div><div><div><img></img><img></img></div><div><div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div><div></div></div></div></div><!--|NAVBAR|--><nav><div><a><img></img></a><a><i></i><span></span></a></div><button><i></i></button

In [79]:
testes_1 = [extrairHtml(teste) for teste in testes]

In [87]:
gerarResultados(jellyfish.jaro_distance, testes_1)

1.00 1.00 0.91 0.85 0.90 0.88 0.77 0.77 0.80 0.76 0.75 0.74 

1.00 1.00 0.91 0.85 0.90 0.88 0.77 0.77 0.80 0.76 0.75 0.74 

0.91 0.91 1.00 0.84 0.90 0.89 0.77 0.77 0.80 0.76 0.75 0.74 

0.85 0.85 0.84 1.00 0.86 0.85 0.85 0.85 0.80 0.84 0.84 0.83 

0.90 0.90 0.90 0.86 1.00 0.92 0.78 0.78 0.80 0.77 0.76 0.75 

0.88 0.88 0.89 0.85 0.92 1.00 0.76 0.76 0.78 0.76 0.75 0.74 

0.77 0.77 0.77 0.85 0.78 0.76 1.00 0.96 0.82 0.92 0.92 0.91 

0.77 0.77 0.77 0.85 0.78 0.76 0.96 1.00 0.81 0.92 0.93 0.92 

0.80 0.80 0.80 0.80 0.80 0.78 0.82 0.81 1.00 0.81 0.80 0.78 

0.76 0.76 0.76 0.84 0.77 0.76 0.92 0.92 0.81 1.00 0.93 0.92 

0.75 0.75 0.75 0.84 0.76 0.75 0.92 0.93 0.80 0.93 1.00 0.94 

0.74 0.74 0.74 0.83 0.75 0.74 0.91 0.92 0.78 0.92 0.94 1.00 

Similaridade Desejada Média = 0.93
Similaridade Não Desejada Média = 0.82


In [88]:
gerarResultados(jellyfish.hamming_distance, testes_1)

0.00 0.00 2313.00 2329.00 2119.00 2202.00 3837.00 3843.00 3824.00 3832.00 3891.00 3882.00 

0.00 0.00 2313.00 2329.00 2119.00 2202.00 3837.00 3843.00 3824.00 3832.00 3891.00 3882.00 

2313.00 2313.00 0.00 2375.00 2289.00 2317.00 3856.00 3860.00 3828.00 3854.00 3874.00 3905.00 

2329.00 2329.00 2375.00 0.00 2029.00 2289.00 2483.00 2495.00 3219.00 2481.00 2529.00 2544.00 

2119.00 2119.00 2289.00 2029.00 0.00 2065.00 3616.00 3608.00 3591.00 3606.00 3662.00 3655.00 

2202.00 2202.00 2317.00 2289.00 2065.00 0.00 3884.00 3877.00 3851.00 3884.00 3929.00 3928.00 

3837.00 3837.00 3856.00 2483.00 3616.00 3884.00 0.00 637.00 2901.00 1062.00 1026.00 1038.00 

3843.00 3843.00 3860.00 2495.00 3608.00 3877.00 637.00 0.00 2918.00 1015.00 968.00 986.00 

3824.00 3824.00 3828.00 3219.00 3591.00 3851.00 2901.00 2918.00 0.00 2892.00 2930.00 2937.00 

3832.00 3832.00 3854.00 2481.00 3606.00 3884.00 1062.00 1015.00 2892.00 0.00 987.00 1013.00 

3891.00 3891.00 3874.00 2529.00 3662.00 3929.00 1026.00 968.0

In [90]:
gerarResultados(jellyfish.jaro_winkler, testes_1)

1.00 1.00 0.95 0.91 0.94 0.93 0.86 0.86 0.88 0.86 0.85 0.85 

1.00 1.00 0.95 0.91 0.94 0.93 0.86 0.86 0.88 0.86 0.85 0.85 

0.95 0.95 1.00 0.91 0.94 0.93 0.86 0.86 0.88 0.86 0.85 0.84 

0.91 0.91 0.91 1.00 0.92 0.91 0.91 0.91 0.88 0.91 0.90 0.90 

0.94 0.94 0.94 0.92 1.00 0.95 0.87 0.87 0.88 0.86 0.86 0.85 

0.93 0.93 0.93 0.91 0.95 1.00 0.86 0.86 0.87 0.86 0.85 0.84 

0.86 0.86 0.86 0.91 0.87 0.86 1.00 0.98 0.89 0.95 0.95 0.95 

0.86 0.86 0.86 0.91 0.87 0.86 0.98 1.00 0.89 0.95 0.96 0.95 

0.88 0.88 0.88 0.88 0.88 0.87 0.89 0.89 1.00 0.89 0.88 0.87 

0.86 0.86 0.86 0.91 0.86 0.86 0.95 0.95 0.89 1.00 0.96 0.95 

0.85 0.85 0.85 0.90 0.86 0.85 0.95 0.96 0.88 0.96 1.00 0.97 

0.85 0.85 0.84 0.90 0.85 0.84 0.95 0.95 0.87 0.95 0.97 1.00 

Similaridade Desejada Média = 0.96
Similaridade Não Desejada Média = 0.89


## HTML similarity

In [91]:
gerarResultados(structural_similarity, testes_1)

1.00 1.00 0.99 0.74 0.69 0.67 0.66 0.67 0.61 0.62 0.66 0.68 

1.00 1.00 0.99 0.74 0.69 0.67 0.66 0.67 0.61 0.62 0.66 0.68 

0.99 0.99 1.00 0.75 0.69 0.67 0.66 0.67 0.61 0.62 0.66 0.68 

0.74 0.74 0.75 1.00 0.76 0.73 0.81 0.82 0.73 0.77 0.85 0.87 

0.69 0.69 0.69 0.76 1.00 0.76 0.65 0.65 0.60 0.62 0.67 0.69 

0.67 0.67 0.67 0.73 0.76 1.00 0.62 0.63 0.57 0.59 0.64 0.66 

0.67 0.67 0.67 0.83 0.67 0.64 1.00 0.99 0.90 0.81 0.88 0.91 

0.68 0.68 0.68 0.84 0.67 0.64 0.99 1.00 0.89 0.82 0.89 0.92 

0.62 0.62 0.62 0.74 0.61 0.59 0.90 0.89 1.00 0.73 0.79 0.81 

0.62 0.62 0.62 0.77 0.62 0.59 0.81 0.82 0.73 1.00 0.84 0.88 

0.66 0.66 0.66 0.85 0.67 0.64 0.88 0.89 0.79 0.84 1.00 0.95 

0.68 0.68 0.68 0.87 0.69 0.66 0.91 0.92 0.81 0.87 0.95 1.00 

Similaridade Desejada Média = 0.93
Similaridade Não Desejada Média = 0.72


In [64]:
gerarResultados(similarity, testes_1)

1.00 1.00 1.00 0.87 0.85 0.83 0.83 0.83 0.83 0.81 0.83 0.84 

1.00 1.00 1.00 0.87 0.85 0.83 0.83 0.83 0.83 0.81 0.83 0.84 

1.00 1.00 1.00 0.87 0.85 0.83 0.83 0.83 0.83 0.81 0.83 0.84 

0.87 0.87 0.87 1.00 0.88 0.86 0.91 0.91 0.91 0.89 0.92 0.93 

0.85 0.85 0.85 0.88 1.00 0.88 0.83 0.83 0.83 0.81 0.84 0.84 

0.83 0.83 0.83 0.86 0.88 1.00 0.81 0.81 0.81 0.80 0.82 0.83 

0.84 0.84 0.84 0.91 0.83 0.82 1.00 1.00 1.00 0.91 0.94 0.96 

0.84 0.84 0.84 0.92 0.83 0.82 1.00 1.00 1.00 0.91 0.94 0.96 

0.84 0.84 0.84 0.91 0.83 0.82 1.00 1.00 1.00 0.91 0.94 0.96 

0.81 0.81 0.81 0.89 0.81 0.80 0.91 0.91 0.91 1.00 0.92 0.94 

0.83 0.83 0.83 0.92 0.84 0.82 0.94 0.94 0.94 0.92 1.00 0.97 

0.84 0.84 0.84 0.93 0.84 0.83 0.96 0.96 0.96 0.94 0.97 1.00 



In [92]:
def extrairHtmlCompleto(url):

    driver = webdriver.Chrome(options=options)
    driver.get(url)
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
    try:
        soup = BeautifulSoup(driver.page_source, "lxml")
        driver.close()
        
        html = str(soup)
            
        return html
    
    except:
        driver.close()
        print("Erro ao tentar abrir a pagina: " + url)

In [66]:
testes_2 = [extrairHtmlCompleto(teste) for teste in testes]

In [93]:
gerarResultados(structural_similarity, testes_2)

1.00 1.00 0.99 0.74 0.69 0.67 0.66 0.67 0.66 0.62 0.66 0.68 

1.00 1.00 0.99 0.74 0.69 0.67 0.66 0.67 0.66 0.62 0.66 0.68 

0.99 0.99 1.00 0.75 0.69 0.67 0.66 0.67 0.66 0.62 0.66 0.68 

0.74 0.74 0.75 1.00 0.76 0.73 0.81 0.82 0.81 0.77 0.85 0.87 

0.69 0.69 0.69 0.76 1.00 0.76 0.65 0.65 0.65 0.62 0.67 0.69 

0.67 0.67 0.67 0.73 0.76 1.00 0.62 0.63 0.62 0.59 0.64 0.66 

0.67 0.67 0.67 0.83 0.67 0.64 1.00 0.99 1.00 0.81 0.88 0.91 

0.68 0.68 0.68 0.84 0.67 0.64 0.99 1.00 0.99 0.82 0.89 0.92 

0.67 0.67 0.67 0.83 0.67 0.64 1.00 0.99 1.00 0.81 0.88 0.91 

0.62 0.62 0.62 0.77 0.62 0.59 0.81 0.82 0.81 1.00 0.84 0.88 

0.66 0.66 0.66 0.85 0.67 0.64 0.88 0.89 0.88 0.84 1.00 0.95 

0.68 0.68 0.68 0.87 0.69 0.66 0.91 0.92 0.91 0.87 0.95 1.00 

Similaridade Desejada Média = 0.94
Similaridade Não Desejada Média = 0.73


In [68]:
gerarResultados(style_similarity, testes_2)

1.00 1.00 0.99 0.76 0.88 0.88 0.53 0.53 0.53 0.51 0.54 0.56 

1.00 1.00 0.99 0.76 0.88 0.88 0.53 0.53 0.53 0.51 0.54 0.56 

0.99 0.99 1.00 0.75 0.87 0.88 0.52 0.52 0.52 0.51 0.54 0.55 

0.76 0.76 0.75 1.00 0.85 0.86 0.65 0.65 0.65 0.66 0.70 0.72 

0.88 0.88 0.87 0.85 1.00 0.99 0.57 0.57 0.57 0.57 0.61 0.63 

0.88 0.88 0.88 0.86 0.99 1.00 0.58 0.58 0.58 0.58 0.61 0.63 

0.53 0.53 0.52 0.65 0.57 0.58 1.00 0.93 1.00 0.76 0.81 0.84 

0.53 0.53 0.52 0.65 0.57 0.58 0.93 1.00 0.93 0.76 0.81 0.84 

0.53 0.53 0.52 0.65 0.57 0.58 1.00 0.93 1.00 0.76 0.81 0.84 

0.51 0.51 0.51 0.66 0.57 0.58 0.76 0.76 0.76 1.00 0.94 0.90 

0.54 0.54 0.54 0.70 0.61 0.61 0.81 0.81 0.81 0.94 1.00 0.96 

0.56 0.56 0.55 0.72 0.63 0.63 0.84 0.84 0.84 0.90 0.96 1.00 



In [94]:
gerarResultados(similarity, testes_2)

1.00 1.00 0.99 0.75 0.78 0.78 0.59 0.60 0.59 0.57 0.60 0.62 

1.00 1.00 0.99 0.75 0.78 0.78 0.59 0.60 0.59 0.57 0.60 0.62 

0.99 0.99 1.00 0.75 0.78 0.77 0.59 0.60 0.59 0.56 0.60 0.62 

0.75 0.75 0.75 1.00 0.81 0.79 0.73 0.73 0.73 0.71 0.77 0.79 

0.78 0.78 0.78 0.81 1.00 0.88 0.61 0.61 0.61 0.60 0.64 0.66 

0.78 0.78 0.77 0.79 0.88 1.00 0.60 0.60 0.60 0.59 0.63 0.64 

0.60 0.60 0.60 0.74 0.62 0.61 1.00 0.96 1.00 0.79 0.84 0.88 

0.60 0.60 0.60 0.74 0.62 0.61 0.96 1.00 0.96 0.79 0.85 0.88 

0.60 0.60 0.60 0.74 0.62 0.61 1.00 0.96 1.00 0.79 0.84 0.88 

0.57 0.57 0.56 0.71 0.60 0.59 0.79 0.79 0.79 1.00 0.89 0.89 

0.60 0.60 0.60 0.77 0.64 0.63 0.84 0.85 0.84 0.89 1.00 0.95 

0.62 0.62 0.62 0.79 0.66 0.64 0.88 0.88 0.88 0.89 0.95 1.00 

Similaridade Desejada Média = 0.95
Similaridade Não Desejada Média = 0.71


## Extração de botões??

In [70]:
def extrairBotoes(url):

    resultados = ""
    
    page = requests.get(url)
    data = page.text
    soup = BeautifulSoup(data, 'html.parser')
    
    buttons = soup.find_all('button')
    for button in buttons:
        resultados+=str(button) + "\n"
        
    return resultados

In [71]:
testes_3 = [extrairBotoes(teste) for teste in testes]

In [95]:
gerarResultados(similarity, testes_3)

1.00 1.00 0.90 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 

1.00 1.00 0.90 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 0.61 

0.90 0.90 1.00 0.55 0.55 0.55 0.55 0.55 0.55 0.55 0.55 0.55 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

0.61 0.61 0.55 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 

Similaridade Desejada Média = 0.98
Similaridade Não Desejada Média = 0.81


In [74]:
def extrairBotoesSelenium(url):

    resultados = ""

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')
    buttons = soup.find_all('button')
    for button in buttons:
        resultados+=str(button)+"\n"
        
    return resultados

In [75]:
testes_4 = [extrairBotoesSelenium(teste) for teste in testes]

In [96]:
gerarResultados(similarity, testes_4)

1.00 1.00 0.95 0.72 0.84 0.84 0.31 0.31 0.20 0.31 0.31 0.31 

1.00 1.00 0.95 0.72 0.84 0.84 0.31 0.31 0.20 0.31 0.31 0.31 

0.95 0.95 1.00 0.69 0.79 0.79 0.29 0.29 0.19 0.29 0.29 0.29 

0.72 0.72 0.69 1.00 0.86 0.86 0.44 0.44 0.24 0.44 0.44 0.44 

0.84 0.84 0.79 0.86 1.00 1.00 0.37 0.37 0.22 0.37 0.37 0.37 

0.84 0.84 0.79 0.86 1.00 1.00 0.37 0.37 0.22 0.37 0.37 0.37 

0.31 0.31 0.29 0.44 0.37 0.37 1.00 1.00 0.23 1.00 1.00 1.00 

0.31 0.31 0.29 0.44 0.37 0.37 1.00 1.00 0.23 1.00 1.00 1.00 

0.25 0.25 0.24 0.25 0.25 0.25 0.23 0.23 1.00 0.23 0.23 0.23 

0.31 0.31 0.29 0.44 0.37 0.37 1.00 1.00 0.23 1.00 1.00 1.00 

0.31 0.31 0.29 0.44 0.37 0.37 1.00 1.00 0.23 1.00 1.00 1.00 

0.31 0.31 0.29 0.44 0.37 0.37 1.00 1.00 0.23 1.00 1.00 1.00 

Similaridade Desejada Média = 0.86
Similaridade Não Desejada Média = 0.51


## Essa métrica de considerar os botões geraram alto nivel de diferença entre páginas de grupos diferentes, mas a similaridade desejada foi bem baixa em alguns casos. E se misturarmos essa métrica com uma de similaridade desejada média alta?

In [104]:
similares = 0
nao_similares = 0

for i in range(len(testes_4)):
    for j in range(len(testes_4)):
        resultado_1 = similarity(testes_4[i], testes_4[j])
        resultado_2 =  similarity(testes_2[i], testes_2[j])
        resultado = (resultado_1+resultado_2)/2
        string = f"{resultado:.2f}"
        if i<3 and j< 3:
            print(GREEN + string, end = " ")
            similares += resultado
        elif i<6 and i>2 and j<6 and j>2:
            print(BLUE + string, end = " ")
            similares += resultado
        elif i<9 and i>5 and j<9 and j>5:
            print(CYAN + string, end = " ")
            similares += resultado
        else:
            print(RED + string, end = " ")
            nao_similares += resultado
    print("\n")

print(RESET + "Similaridade Desejada Média = " + f"{(similares/27):.2f}")
print("Similaridade Não Desejada Média = " + f"{(nao_similares/117):.2f}")

1.00 1.00 0.97 0.74 0.81 0.81 0.45 0.45 0.40 0.44 0.45 0.46 

1.00 1.00 0.97 0.74 0.81 0.81 0.45 0.45 0.40 0.44 0.45 0.46 

0.97 0.97 1.00 0.72 0.79 0.78 0.44 0.44 0.39 0.43 0.44 0.45 

0.74 0.74 0.72 1.00 0.83 0.83 0.58 0.58 0.48 0.58 0.60 0.62 

0.81 0.81 0.79 0.83 1.00 0.94 0.49 0.49 0.42 0.48 0.51 0.51 

0.81 0.81 0.78 0.83 0.94 1.00 0.49 0.49 0.41 0.48 0.50 0.51 

0.45 0.45 0.44 0.59 0.50 0.49 1.00 0.98 0.61 0.89 0.92 0.94 

0.45 0.45 0.44 0.59 0.50 0.49 0.98 1.00 0.59 0.89 0.92 0.94 

0.42 0.42 0.42 0.50 0.44 0.43 0.61 0.59 1.00 0.51 0.54 0.55 

0.44 0.44 0.43 0.58 0.48 0.48 0.89 0.89 0.51 1.00 0.95 0.95 

0.45 0.45 0.44 0.60 0.51 0.50 0.92 0.92 0.54 0.95 1.00 0.98 

0.46 0.46 0.45 0.62 0.51 0.51 0.94 0.94 0.55 0.94 0.98 1.00 

Similaridade Desejada Média = 0.91
Similaridade Não Desejada Média = 0.61
